<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza y Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano y Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: FELIPE GALAZ GARCIA
- Nombre de alumno 2: MIRKO MARTINIC JARA


### **Link de repositorio de GitHub:** `https://github.com/mmartinicj/MDS7202-labs`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/15.6 MB 25.8 MB/s eta 0:00:01
   ------ --------------------------------- 2.5/15.6 MB 32.3 MB/s eta 0:00:01
   ------------ --------------------------- 5.0/15.6 MB 40.1 MB/s eta 0:00:01
   ---------------------- ----------------- 8.7/15.6 MB 50.6 MB/s eta 0:00:01
   --------------------------------- ------ 13.3/15.6 MB 81.8 MB/s eta 0:00:01
   ---------------------------------------  15.6/15.6 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------- 15.6/15.6 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.17.0
    Uninstalling plotly-5.17.0:
      Successfully uninstalled plotly-5.17.0



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.csv`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [4]:
df_players = pd.read_csv('Lab_08/stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [5]:
### Código aquí ###

# agregamos columna label (SELECCIONADO == 1 | NO SELECCIONADO == 0)
df_players['label'] = np.where(df_players.National_Position.isna(), 0, 1)

# obtenemos la distribucion del target
df_players.label.value_counts()

label
0    16513
1     1075
Name: count, dtype: int64

In [17]:
n_min = df_players.label.value_counts()[1]
n_max = df_players.label.value_counts()[0] 

print(f"El número de jugadores seleccionados es {n_min}, mientras que los no seleccionados ascienden a {n_max}.\nCon lo anterior, el % de seleccionados asciende a {round(n_min/n_max*100, 2)}%.")

El número de jugadores seleccionados es 1075, mientras que los no seleccionados ascienden a 16513.
Con lo anterior, el % de seleccionados asciende a 6.51%.


En virtud del bloque de código precedente, se observa que se tienen 16513 ejemplos de la clase **JUGADORES NO SELECCIONADOS** y 1075 ejemplos de la clase **JUGADORES SELECCIONADOS**, por lo cual, se reconoce un problema de desbalance de clases, donde la categoría minoritaria representa aproximadamente un 6 % respecto de la categoría mayoritaria. Así, es importante que dicho problema sea considerado al momento de evaluar el desempeño de los modelos de clasificación que sean implementados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [18]:
### Código aquí ###

# Genere un ColumnTransformer enfocado en preprocesar los datos.
drop_cols = ['Name', 'Nationality', 'National_Position']
cat_cols = ['Club_Position', 'Preffered_Foot', 'Work_Rate', 'Weak_foot', 'Skill_Moves']
num_cols = [c for c in df_players.columns if c not in drop_cols+cat_cols and c != 'label']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_preprocess', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols),
        ('num_preprocess', MinMaxScaler(), num_cols)
    ], remainder='drop'
)

# Crear 3 pipelines con diferentes clasificadores.
# pipeline para el 1°clasificador
from sklearn.svm import LinearSVC
clf_1 = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", LinearSVC(random_state=42))]
)
# pipeline para el 2°clasificador
from sklearn.neighbors import KNeighborsClassifier
clf_2 = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", KNeighborsClassifier())]
)
# pipeline para el 3°clasificador
from sklearn.ensemble import BaggingClassifier
clf_3 = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42)))]
)

En virtud del bloque de código precedente, se deben notar los siguientes aspectos:

- Se han escogido las transformaciones `OneHotEncoder` y `MinMaxScaler` para preprocesar, respectivamente, los atributos categóricos y numéricos, en donde los atributos categóricos corresponden a las variables `Club_Position`, `Preffered_Foot`, `Work_Rate`, `Weak_foot` y `Skill_Moves`, mientras que el resto de las variables corresponden a los atributos numéricos:

    - La transformación `OneHotEncoder`, por un lado, permite codificar atributos categóricos como una matriz numérica única ("one-hot numeric array"). En el ámbito del problema en estudio, se aplicó esta transformación fijando `drop='first'` para prevenir situaciones de multicolinealidad entre las variables y `handle_unknown='ignore'` para omitir aquellas categorías que no aparezcan en el conjunto de entrenamiento.
    
    - La transformación `MinMaxScaler`, por otro lado, permite escalar los atributos numéricos al rango [0,1]. En el ámbito del problema en estudio, se aplico esta transformación dado que los modelos de clasifiación implementados más adelante se desempeñan mejor cuando las variables se encuentran en una escala similar y/o distribuyen normal. No obstante lo anterior, a partir de la revisión del material `Player_Stats_Report.html`, se reconoce, en términos generales, que las variables numéricas distribuyen normal unimodal/bimodal, por lo que, sólo restaba unificar la escala entre ellas.


- Cabe señalar que la variable `National_Position` es descartada del problema de clasificación puesto que a partir de esta se generaron los labels. Asimismo, las variables `Name` y `Nationality` también son descartadas debido a que son considerados como atributos identificadores de los jugadores y que, *a priori*, no aportarían información relevante sobre la predicción de las etiquetas.


- Por último, dado que (1) se tienen > 50 muestras, (2) estamos prediciendo categorías, (3) disponemos de etiquetas para la data y (4) el tamaño del dataset es < 100k muestras, entonces se construyen pipelines para tres modelos de clasificación: `Linear SVC`, `KNeighbors Classifier` y `Ensemble Classifier` (específicamente `Bagging Classifier` de árboles de decisión). 

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [19]:
### Código aquí ###

# Separar el conjunto de datos en entrenamiento y de prueba.
X = df_players.drop('label', axis=1)
y = df_players.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Entrenar los pipelines.
model_1 = clf_1.fit(X_train, y_train)
model_2 = clf_2.fit(X_train, y_train)
model_3 = clf_3.fit(X_train, y_train)

# Utilizar las métricas para evaluar los modelos generados.
print('- \033[1mDESEMPEÑO MODELO 1 | LinearSVC\033[0m \n')
print(classification_report(y_test, model_1.predict(X_test)))
print('- \033[1mDESEMPEÑO MODELO 2 | KNeighborsClassifier\033[0m \n')
print(classification_report(y_test, model_2.predict(X_test)))
print('- \033[1mDESEMPEÑO MODELO 3 | BaggingClassifier\033[0m \n')
print(classification_report(y_test, model_3.predict(X_test)))

- DESEMPEÑO MODELO 1 | LinearSVC 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.67      0.01      0.01       323

    accuracy                           0.94      5277
   macro avg       0.80      0.50      0.49      5277
weighted avg       0.92      0.94      0.91      5277

- DESEMPEÑO MODELO 2 | KNeighborsClassifier 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.32      0.03      0.06       323

    accuracy                           0.94      5277
   macro avg       0.63      0.51      0.51      5277
weighted avg       0.90      0.94      0.91      5277

- DESEMPEÑO MODELO 3 | BaggingClassifier 

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      4954
           1       0.38      0.07      0.12       323

    accuracy                           0.94      5277
   m

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

1. ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?

**R.** El accuracy obtenido es bastante alto para los tres modelos (sobre el 90 %), sin embargo, dado el problema de desbalance de clases señalado inicialmente, adquieren mayor importancia las métricas precisión y recall, cuyos valores se encuentran, respectivamente, en los rangos 32-67 % y 1-7 %, siendo el modelo `LinearSVC` aquel con mejor desempeño para la predicción de la clase positiva.

2. ¿Podrían mejorarse los resultados?, ¿Cómo?

**R.** Los resultados obtenidos podrían ser mejorados seleccionando aquel modelo con mejor desempeño (correspondiente al modelo `LinearSVC`) y, luego, realizando una búsqueda de los hiperparámetros óptimos mediante algún algoritmo pertinente (*e.g.* `GridSearchCV` de la librería `sklearn`). Durante dicha búsqueda, podrían evaluarse, por ejemplo, los hiperparámetros `penalty`, `tol` y `C`, entre otros, de manera tal que, con el modelo optimizado, se obtengan mejores métricas de desempeño.

3. ¿Influye la cantidad de ejemplos por clase?

**R.** En términos generales, los modelos tipo SVM (*e.g* `LinearSVC`) y ensemble (*e.g.* `BaggingClassifier`) tienden a no ser afectados por el desbalance de clases debido a la formulación teórica asociada a estos, mientras que modelos como `KNeighborsClassifier` sí son afectados como se observa en el bloque de código precedente, en donde este modelo obtiene el peor desempeño para la predicción de la clase positiva. Así, se concluye que la cantidad de ejemplos por clase SÍ influye en los resultados obtenidos dependiendo del tipo de modelo de clasificación utilizado.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

**Respuesta:**

In [21]:
### Código aquí ###

df_players = pd.read_csv('Lab_08/stats_players.csv')

#  Renacín les recalca que no deben utilizar los valores Sub y Res de esta columna.
df_players = df_players[~df_players.Club_Position.isin(['Sub', 'Res'])] 

# Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
club_position_map = {'ataque':['ST', 'CF'], 'central_ataque':['RW', 'CAM', 'LW'], 'central':['RM', 'CM', 'LM'],
                     'central_defensa':['RWB', 'CDM', 'LWB'], 'defensa':['RB', 'CB', 'LB'], 'arquero':['GK']}
df_players.Club_Position = df_players.Club_Position.map({j: i for i in club_position_map for j in club_position_map[i]})
df_players = df_players.dropna(subset=['Club_Position']).reset_index(drop=True)

# Cuente cuantos por clase quedan.
print('- \033[1mCANTIDAD DE CLASES EN Club_Position\033[0m')
display(df_players.Club_Position.value_counts())

# Entrene el nuevo pipeline y ejecute una evaluación de este.

# - Genere un ColumnTransformer enfocado en preprocesar los datos
drop_cols = ['Name', 'Nationality', 'National_Position']
cat_cols = ['Preffered_Foot', 'Work_Rate', 'Weak_foot', 'Skill_Moves']
num_cols = [c for c in df_players.columns if c not in drop_cols+cat_cols and c != 'Club_Position']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_preprocess', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols),
        ('num_preprocess', MinMaxScaler(), num_cols)
    ], remainder='drop'
)

# - Crear pipelines
clf = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", LinearSVC(random_state=42))]
)

# - Separar el conjunto de datos en entrenamiento y de prueba.
X = df_players.drop('Club_Position', axis=1)
y = pd.factorize(df_players.Club_Position)[0] + 1 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# - Entrenar los pipelines.
model = clf.fit(X_train, y_train)

# - Utilizar las métricas para evaluar los modelos generados.
print('\n- \033[1mREPORTE DE CLASIFICACION | LinearSVC\033[0m \n')
print(classification_report(y_test, model.predict(X_test)))

- CANTIDAD DE CLASES EN Club_Position


Club_Position
defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: count, dtype: int64


- REPORTE DE CLASIFICACION | LinearSVC 

              precision    recall  f1-score   support

           1       0.56      0.32      0.40       174
           2       0.77      0.90      0.83       129
           3       1.00      1.00      1.00       190
           4       0.82      0.98      0.89       354
           5       0.61      0.68      0.65       272
           6       0.58      0.17      0.27        63

    accuracy                           0.76      1182
   macro avg       0.72      0.68      0.67      1182
weighted avg       0.74      0.76      0.74      1182



**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

En virtud del bloque de código precedente, se deben notar los siguientes aspectos:

- Se han escogido las transformaciones `OneHotEncoder` y `MinMaxScaler` para preprocesar, respectivamente, los atributos categóricos y numéricos, en donde los atributos categóricos corresponden a las variables `Preffered_Foot`, `Work_Rate`, `Weak_foot` y `Skill_Moves`, mientras que el resto de las variables corresponden a los atributos numéricos. 


- Cabe señalar que la variable `National_Position` es descartada del problema de clasificación puesto que tiene una cantidad significativa de valores nulos (> 90 %). Asimismo, las variables `Name` y `Nationality` también son descartadas debido a que son considerados como atributos identificadores de los jugadores y que, *a priori*, no aportarían información relevante sobre la predicción de las etiquetas.


- En cuanto a la variable `Club_Position`, esta fue re-codificada según las indicaciones y transformada a una variable numérica con la función `factorize`, obteniéndose, así, las etiquetas para el problema de clasificación. Se debe notar que esta variable también exhibe un problema de desbalance de clases (*e.g.* la categoría `defensa` tiene 1180 ejemplos, mientras que la categoría `central_defensa` posee 209 ejemplos). 


- Dado que (1) se tienen > 50 muestras, (2) estamos prediciendo categorías, (3) disponemos de etiquetas para la data y (4) el tamaño del dataset es < 100k muestras, entonces se construye un pipeline para el modelo de clasificación `Linear SVC`, cuyos resultados pueden ser considerados satisfactorios, pues, el modelo permite predecir correctamente la mayoría de las etiquetas con un F1-score sobre el 60 %, excepto para las etiquetas `central_ataque` y `central_defensa`.


- Teniendo en consideración el problema del modelo implementado para la predicción de las etiquetas `central_ataque` y `central_defensa`, podría realizarse una búsqueda de hiperparámetros óptimos (*e.g.* `penalty`, `tol` y `C`) que, en última instancia, permitan obtener mejores métricas de desempeño con el modelo optimizado.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [X] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [X] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [25]:
### Código aquí ###

df_players = pd.read_csv('Lab_08/stats_players.csv')

# unificamos los datasets de 'salarios.csv' y 'stats_players.csv'
df_salary = pd.read_csv('Lab_08/salarios.csv').drop('Unnamed: 0', axis=1).merge(df_players, left_on='Player', right_on='Name')
df_salary['Weekly_Salary'] = np.log(df_salary['Weekly Salary'])
df_salary.drop(['Player', 'Weekly Salary'], axis=1, inplace=True)

# se convierte National_Position para indicar si el jugador es seleccionado nacional
df_salary['seleccionado'] = np.where(df_salary.National_Position.isna(), 0, 1)

# visualizamos la distribucion de la variable objetivo
fig = px.histogram(np.exp(df_salary.Weekly_Salary), 
                   marginal="box", barmode='group')
fig.update_layout(
    title=f"DISTRIBUCIÓN DEL SALARIO SEMANAL",
    xaxis_title="Weekly Salary",
    yaxis_title="Count")
fig.show()

# Generar un pipeline para la regresión.
# Obtener un regresor con un  R2 superior a  0.35.

# - Genere un ColumnTransformer enfocado en preprocesar los datos
drop_cols = ['Name', 'Nationality', 'National_Position']
cat_cols = ['Club_Position', 'Preffered_Foot', 'Work_Rate', 'Weak_foot', 'Skill_Moves' ,'seleccionado']
num_cols = [c for c in df_salary.columns if c not in drop_cols+cat_cols and c != 'Weekly_Salary']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_preprocess', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols),
        ('num_preprocess', MinMaxScaler(), num_cols)
    ], remainder='drop'
)

# - Crear pipelines
from sklearn.linear_model import SGDRegressor
reg = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("regressor", SGDRegressor(max_iter=1000, tol=1e-3))]
)

# - Separar el conjunto de datos en entrenamiento y de prueba.
X = df_salary.drop('Weekly_Salary', axis=1)
y = np.array(df_salary.Weekly_Salary)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# - Entrenar los pipelines.
model = reg.fit(X_train, y_train)

# - Utilizar las métricas para evaluar los modelos generados.
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
fig = px.scatter(x=y_test, y=y_pred, trendline="ols")
fig.add_shape(type="line", x0=6, y0=6, x1=15, y1=15)
fig.update_layout(
    title=f"DESEMPEÑO MODELO R2 = {r2_score(y_test, y_pred):.2f}",
    xaxis_title="Real Weekly Salary",
    yaxis_title="Predicted Weekly Salary")
fig.show()

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

En virtud del bloque de código precedente, se deben notar los siguientes aspectos:

- La métrica R2 constituye una medida estadística de un modelo de regresión que determina la proporción de varianza en la variable dependiente que puede explicarse por la/s variable/s independiente/s. Dicha métrica puede tener valores en el rango [0,1], donde 1 significa un ajuste lineal perfecto.


- Se han escogido las transformaciones `OneHotEncoder` y `MinMaxScaler` para preprocesar, respectivamente, los atributos categóricos y numéricos, en donde los atributos categóricos corresponden a las variables `Club_Position`, `Preffered_Foot`, `Work_Rate`, `Weak_foot` y `Skill_Moves`, mientras que el resto de las variables corresponden a los atributos numéricos. Se realizó una transformación adicional, correspondiente a una transformación logarítmica, para la variable `Weekly_Salary`. Lo anterior a que esta exhibe una distribución no normal con una marcada asimetría positiva y que puede influir sobre la distribución de los residuos del modelo, los cuales de preferencia se espera que tengan una distribución lo más cercana posible a una normal.


- Cabe señalar que la variable `National_Position` es recodificada para reflejar el estado como seleccionado nacional del jugador en la variable `seleccionado`. Lo anterior dada la suposición de que el sueldo del jugador puede ser explicado, parcialmente, por si es o no seleccionado nacional.

- Las variables `Name` y `Nationality` son descartadas debido a que son considerados como atributos identificadores de los jugadores y que, *a priori*, no aportarían información relevante sobre la predicción de los salarios.


- Dado que (1) se tienen > 50 muestras, (2) estamos prediciendo cantidades y (4) el tamaño del dataset es < 100k muestras, entonces se construye un pipeline para el modelo de regresión `SGD Regressor`, obteniéndose un R2 de 40 %, sólo un poco por sobre el mínimo esperado. Dado lo anterior, es posible sostener que el desempeño es más bien mediocre y sería útil contar con uno mejor, especialmente si el objetivo es entender mejor de qué dependen los sueldos de los jugadores.


- Finalmente, para mejorar los resultados del modelo se considera necesario obtener más datos debido a que, al realizar la unión de los datasets `salarios.csv` y `stats_players.csv`, la cantidad de datos se reduce desde 17588 a 1866 *i.e.* una reducción de aproximadamente el 90 %. Así, con una mayor cantidad de datos, podrían reajustarse los valores de los parámetros del modelo `SGD Regressor`, resultando, por tanto, en un mejor desempeño para la predicción del salario.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>